# Potential Outcomes Causal Model

My own reworking of the example code within the text book.

In [1]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
import altair as alt
from itertools import combinations 

### Create a random test score dataset

In [2]:
np.random.seed(22)

test_scores = pd.DataFrame(
    {'score': np.random.normal(50, 25, 1000)}
)

# Select only scores between 0 and 100
test_scores = test_scores[
    (test_scores['score'] >= 0) 
  & (test_scores['score'] <= 100)
].reset_index(drop=True)

# Check this worked
assert sum(test_scores['score'] < 0) == 0, 'Some scores are less than zero!'
assert sum(test_scores['score'] > 100) == 0, 'Some scores are greater than one hundred!'

### Assume our cutoff ($c$) is when test score is 50 and assign units above and below this threshold to different groups

In [3]:
test_scores['D'] = 0
test_scores.loc[test_scores['score'] > 50, 'D'] = 1

### Add data with an effect ($Y_1$) and without an effect ($Y_2$)

Remember, the regression equation looks like the below...

$$
\Large Y_i =\alpha+\beta X_i + \delta D_i + \varepsilon_i
$$

In [4]:
test_scores['y1'] = (
    25 
  + 1.5 * test_scores.score
  + 0*test_scores.D  # Remove relationship with assignment
  + np.random.normal(0, 20, test_scores.shape[0])
)
            
test_scores['y2'] = (
    25 
  + 1.5 * test_scores.score 
  + 50*test_scores.D 
  + np.random.normal(0, 20, test_scores.shape[0])
)

### Create plot showing null model with no effect

In [5]:
chart = alt.Chart(
    test_scores,
    title="No Effect (smooth continuity)"
).mark_point(opacity=.5).encode(
    x = alt.X("score:Q", title = "test scores"),
    y = alt.Y("y1:Q", title = 'outcome variable (y1)'),
    color = alt.Color("D:N",title = ["group","assignment"]),
    tooltip = [
        alt.Tooltip("score:Q", format='.2f'),
        alt.Tooltip("y1:Q", title='outcome var', format='.2f'),
    ]
).properties(width=800, height=500)

# Add regression line for each group
sep_lines = chart + chart.transform_regression("score", "y1", groupby=['D']).mark_line(size=2)

# Add vertrical line
vert_df = pd.DataFrame({
        'x' : [50],
    })
vert = alt.Chart(vert_df).mark_rule(color='black').encode(
    x = 'x:Q',
)

# Add line for all data
# sep_lines + chart.transform_regression("score", "y1").mark_line(size=2, color='black')

final = alt.layer(sep_lines, vert)

# Format plot
final.configure_title(
    fontSize=16,
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_legend(
    titleFontSize=16,
    labelFontSize=14
)

alt.LayerChart(...)

### Create some dataframes for annotation

In [6]:
annotation_df = pd.DataFrame(
    {
        'line' : ['top','top', 'bottom','bottom'],
        'x' : [0, 50, 0, 50],
        'y' : [150,150, 95, 95],
    }
)

text_df = pd.DataFrame(
    {
        'text' : ['Vertical gap = LATE'],
        'x' : [5],
        'y' : [160],
    }
)

### Create plot showing regression discontinuity with causal effect at cutoff ($c=50$)

In [7]:
chart = alt.Chart(
    test_scores,
    title="Causal effect (clear discontinuity)"
).mark_point(opacity=.5).encode(
    x = alt.X("score:Q", title = "test scores"),
    y = alt.Y("y2:Q", title = 'outcome variable (y2)'),
    color = alt.Color("D:N",title = ["group","assignment"]),
    tooltip = [
        alt.Tooltip("score:Q", format='.2f'),
        alt.Tooltip("y2:Q", title='outcome var', format='.2f'),
    ]
).properties(width=800, height=500)

sep_lines = chart + chart.transform_regression("score", "y2", groupby=['D']).mark_line(size=2)

top_line = alt.Chart(
    annotation_df[annotation_df['line']=='top']
).mark_line(color='black', strokeDash=[10,5]).encode(
    x = "x:Q",
    y = 'y:Q'
)

bottom_line = alt.Chart(
    annotation_df[annotation_df['line']=='bottom']
).mark_line(color='black', strokeDash=[10,5]).encode(
    x = "x:Q",
    y = 'y:Q'
)

text = alt.Chart(text_df).mark_text(align='left', fontSize=20, fontWeight='bold').encode(
    x = 'x:Q',
    y = 'y:Q',
    text = 'text:N',
    
)

vert_df = pd.DataFrame({
        'x' : [50],
    })
vert = alt.Chart(vert_df).mark_rule(color='black').encode(
    x = 'x:Q',
)

final = alt.layer(sep_lines, top_line, bottom_line, text, vert)


final.configure_title(
    fontSize=16,
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_legend(
    titleFontSize=16,
    labelFontSize=14
)

alt.LayerChart(...)

### Create non-linear dataset

In [8]:
non_lin_scores = pd.DataFrame(
    {'x': np.random.normal(100, 50, 1000)}
)
non_lin_scores.loc[non_lin_scores.x<0, 'x'] = 0
non_lin_scores['x2'] = non_lin_scores['x']**2
non_lin_scores['x3'] = non_lin_scores['x']**3
non_lin_scores['D'] = 0
non_lin_scores.loc[non_lin_scores.x>140, 'D'] = 1

In [9]:

non_lin_scores['y3'] = (
    10000 
  + 0*non_lin_scores.D 
  - 100 * non_lin_scores.x 
  + non_lin_scores.x2 
  + np.random.normal(0, 1750, 1000)
)
non_lin_scores

,x,x2,x3,D,y3
0,71.784322,5152.988867,3.699038e+05,0,8319.114869
1,123.814357,15329.995032,1.898073e+06,0,14271.286924
2,138.071153,19063.643221,2.632139e+06,0,16309.671589
3,51.335313,2635.314328,1.352847e+05,0,9167.397837
4,119.770644,14345.007248,1.718111e+06,0,12639.167336
...,...,...,...,...,...
995,110.983607,12317.361047,1.367025e+06,0,8972.777543
996,185.253068,34318.699288,6.357644e+06,1,28067.723920
997,140.627313,19776.041124,2.781052e+06,1,18349.770379
998,0.000000,0.000000,0.000000e+00,0,5758.326449


In [10]:
chart = alt.Chart(
    non_lin_scores,
    title="Spurious causal effect"
).mark_point(opacity=.5).encode(
    x = alt.X("x:Q", title = "test scores"),
    y = alt.Y("y3:Q", title = 'outcome variable (y3)'),
    color = alt.Color("D:N",title = ["group","assignment"]),
    tooltip = [
        alt.Tooltip("x:Q", title='score', format='.2f'),
        alt.Tooltip("y3:Q", title='outcome var', format='.2f'),
    ]
).properties(width=800, height=500)

sep_lines = chart + chart.transform_regression("x", "y3", groupby=['D']).mark_line(size=2)

vert_df = pd.DataFrame({
        'x' : [140],
    })
vert = alt.Chart(vert_df).mark_rule(color='black').encode(
    x = 'x:Q',
)

final = alt.layer(sep_lines, vert)
final.configure_title(
    fontSize=16,
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_legend(
    titleFontSize=16,
    labelFontSize=14
)

alt.LayerChart(...)

In [11]:
chart = alt.Chart(
    non_lin_scores,
    title="With LOESS fit"
).mark_point(opacity=.5).encode(
    x = alt.X("x:Q", title = "test scores"),
    y = alt.Y("y3:Q", title = 'outcome variable (y3)'),
    color = alt.Color("D:N",title = ["group","assignment"]),
    tooltip = [
        alt.Tooltip("x:Q", title='score', format='.2f'),
        alt.Tooltip("y3:Q", title='outcome var', format='.2f'),
    ]
).properties(width=800, height=500)

sep_lines = chart + chart.transform_loess("x", "y3", groupby=['D']).mark_line(size=2)

vert_df = pd.DataFrame({
        'x' : [140],
    })
vert = alt.Chart(vert_df).mark_rule(color='black').encode(
    x = 'x:Q',
)

final = alt.layer(sep_lines, vert)
final.configure_title(
    fontSize=16,
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_legend(
    titleFontSize=16,
    labelFontSize=14
)

alt.LayerChart(...)

### Note:
Gelman, and Imbens (2019)$^1$ discouraged the use of higher-order polynomials when estimating local linear regressions. Standard solution is to run local linear nonparametric regression.$^2$

So what is that? This method uses a kernel to weight the regression to a restricted a window (hence “local”). A rectangular kernel would be simliar to taking $E[Y]$ within some bin whereas a triangular kernel gives more importance to the observations closest to the center.

This method is sensitive to the choice of bandwidth, but more recent work allows the researcher to estimate optimal bandwidths.$^{3,4}$

References:
1. Gelman and Imbens. 2019. “Why Higher-Order Polynomials Should Not Be Used in Regression Discontinuity Designs.” Journal of Business and Economic Statistics 37 (3): 447–56.
2. Hahn, Todd, and van der Klaauw. 2001. “Identification and Estimation of Treatment Effects with a Regression-Discontinuity Design.” Econometrica 69 (1): 201–9.
3.  Imbens and Kalyanaraman. 2011. “Optimal Bandwidth Choice for the Regression Discontinuity Estimator.” The Review of Economic Studies 79 (3): 933–59.
4. Calonico, Cattaneo, and Titiunik. 2014. “Robust Nonparametric Confidence Intervals for Regression-Discontinuity Designs.” Econometrica 82 (6): 2295–2326. 
